In [1]:
from selenium import webdriver
import pandas as pd
import os
import datetime
from multiprocessing.pool import ThreadPool
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
chromeOptions = Options()

In [3]:
n_pcu = 1
pool = ThreadPool(n_pcu)
drivers = [webdriver.Chrome(
    service=Service(ChromeDriverManager().install())
) for i in range(n_pcu)]

In [5]:
drivers[0]

<selenium.webdriver.chrome.webdriver.WebDriver (session="61d72e18174bf0442fe876c3b1de3960")>

In [6]:
page = 0
base_url = "https://groww.in/mutual-funds/filter?q=&fundSize=&category=%5B%22Equity%22%5D&pageNo={}&sortBy=3".format(page)

In [7]:
drivers[0].get(base_url)

In [9]:
table = drivers[0].find_element(By.CLASS_NAME, "tb10Table")

In [10]:
table

<selenium.webdriver.remote.webelement.WebElement (session="61d72e18174bf0442fe876c3b1de3960", element="d8ac3ea9-aba0-4b26-99e0-04f70bd88413")>

In [12]:
elements = table.find_elements(By.TAG_NAME, "tr")

In [16]:
elements[0].find_element(By.TAG_NAME, "a").get_attribute("href")

'https://groww.in/mutual-funds/quant-small-cap-fund-direct-plan-growth'

In [22]:
td = elements[0].find_elements(By.TAG_NAME, "td")

In [30]:
td[1].text

'Quant Small Cap Fund Direct Plan Growth\nVery High Risk\nEquity\n5'

In [59]:
def get_page_summary(page):
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install())
    )
    base_url = "https://groww.in/mutual-funds/filter?q=&fundSize=&category=%5B%22Equity%22%5D&pageNo={}&sortBy=3".format(page)
    driver.get(base_url)
    table = driver.find_element(By.CLASS_NAME, "tb10Table")
    elements = table.find_elements(By.TAG_NAME, "tr")
    data = {
        "name":[],
        "link":[],
        "risk":[]
    }
    for element in elements:
        link = element.find_element(By.TAG_NAME, "a").get_attribute("href")
        td = element.find_elements(By.TAG_NAME, "td")
        fund = td[1].text.split("\n")
        data['link'].append(link)
        data["name"].append(fund[0])
        data["risk"].append(fund[1])
    driver.close()
    return pd.DataFrame(data)


In [71]:
date = str(datetime.date.today())
date
root_dir = "../data/grow"
if not os.path.exists(os.path.join(root_dir, date, "index")):
    os.makedirs(os.path.join(root_dir, date, "index"))
    data = get_page_summary(1)
    path = os.path.join(root_dir, date, "index", "{}.csv".format(page))
    data.to_csv(path, index=None)

In [ ]:
data

,name,link,risk
0,Mirae Asset Tax Saver Fund Direct Growth,https://groww.in/mutual-funds/mirae-asset-tax-...,Very High Risk
1,SBI Magnum Mid Cap Direct Plan Growth,https://groww.in/mutual-funds/sbi-magnum-midca...,Very High Risk
2,HDFC Small Cap Fund Direct Growth,https://groww.in/mutual-funds/hdfc-small-cap-f...,Very High Risk
3,SBI Bluechip Direct Plan Growth,https://groww.in/mutual-funds/sbi-bluechip-fun...,Very High Risk
4,Tata Small Cap Fund Direct Growth,https://groww.in/mutual-funds/tata-small-cap-f...,Very High Risk
5,ICICI Prudential Smallcap Fund Direct Plan Growth,https://groww.in/mutual-funds/icici-prudential...,Very High Risk
6,Axis Midcap Direct Plan Growth,https://groww.in/mutual-funds/axis-midcap-fund...,Very High Risk
7,Parag Parikh Tax Saver Fund Direct Growth,https://groww.in/mutual-funds/parag-parikh-tax...,Moderately High Risk
8,Quant Infrastructure Fund Direct Growth,https://groww.in/mutual-funds/quant-infrastruc...,Very High Risk
9,Bank of India Small Cap Fund Direct Growth,https://groww.in/mutual-funds/boi-axa-small-ca...,Very High Risk


In [2]:
pool = ThreadPool(2)
def fund_scraper(url):
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
    )
    driver.get(url)
    element = driver.find_element(By.XPATH, "//div[@class='holdings101Cta cur-po']")
    element.click()

In [4]:
url = ["https://groww.in/mutual-funds/quant-small-cap-fund-direct-plan-growth"]
pool.apply(fund_scraper, url )

In [5]:
fund_scraper(url[0])

In [8]:
driver = webdriver.Chrome(
service=Service(ChromeDriverManager().install()),
    )
driver.get(url[0])
element = driver.find_element(By.XPATH, "//div[@class='holdings101Cta cur-po']")
element.click()

In [16]:
element = driver.find_element(By.XPATH, "//table[@class='tb10Table holdings101Table']")

In [22]:
tbl = element.get_attribute("outerHTML")

In [24]:
df = pd.read_html(tbl)

In [27]:
df[0]

,Name,Sector,Instrument,Assets
0,Reliance Industries Ltd.,Energy,Equity,6.5%
1,ITC Ltd.,Consumer Staples,Equity,5.6%
2,HDFC Bank Ltd.,Financial,Equity,5.3%
3,Jindal Stainless Ltd.,Metals & Mining,Equity,5.2%
4,RBL Bank Ltd.,Financial,Equity,4.5%
...,...,...,...,...
66,PTC India Financial Services Ltd.,Financial,Equity,0.2%
67,NACL Industries Ltd.,Chemicals,Equity,0.2%
68,AGS Transact Technologies Ltd.,Services,Equity,0.2%
69,Nahar Spinning Mills Ltd.,Textiles,Equity,0.1%


In [33]:
def scrape_fund(name,url):
    driver = webdriver.Chrome(
    service=Service(ChromeDriverManager().install()),
        )
    driver.get(url)
    element = driver.find_element(By.XPATH, "//div[@class='holdings101Cta cur-po']")
    element.click()
    element = driver.find_element(By.XPATH, "//table[@class='tb10Table holdings101Table']")
    tbl = element.get_attribute("outerHTML")
    df = pd.read_html(tbl)
    df[0].to_csv("data/grow/2023-04-23/funds/{}.csv".format(name))
    driver.close()
def scrape_funds():
    date = str(datetime.date.today())
    funds_dir = os.path.join("data/grow",date, "funds")
    index_dir = os.path.join("data/grow",date, "index")
    index_path = os.path.join(index_dir, os.listdir(index_dir)[0])
    if os.path.exists(funds_dir):
        return "already up to date"
    else:
        os.mkdir(funds_dir)
        index = pd.read_csv(index_path)[["name", "link"]].values
        # print(index)
        pool = ThreadPool(os.cpu_count())
        results = pool.starmap(scrape_fund,index)


In [34]:
scrape_funds()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@class='holdings101Cta cur-po']"}
  (Session info: chrome=112.0.5615.165)
Stacktrace:
#0 0x5582cca90fe3 <unknown>
#1 0x5582cc7cfd36 <unknown>
#2 0x5582cc80cc4d <unknown>
#3 0x5582cc80cd61 <unknown>
#4 0x5582cc8486a4 <unknown>
#5 0x5582cc82d8ed <unknown>
#6 0x5582cc846232 <unknown>
#7 0x5582cc82d693 <unknown>
#8 0x5582cc80003a <unknown>
#9 0x5582cc80117e <unknown>
#10 0x5582cca52dbd <unknown>
#11 0x5582cca56c6c <unknown>
#12 0x5582cca604b0 <unknown>
#13 0x5582cca57d63 <unknown>
#14 0x5582cca2ac35 <unknown>
#15 0x5582cca7b138 <unknown>
#16 0x5582cca7b2c7 <unknown>
#17 0x5582cca89093 <unknown>
#18 0x7f687f8bc609 start_thread
